In [48]:
import pandas as pd

In [113]:
df = pd.read_csv("data/training_set.csv")

In [114]:
user_list = df.groupby('UserId')['ItemId'].nunique()
user_list_denoise = user_list[(user_list<=50000)].index.to_list()

In [115]:
test = pd.read_csv('data/public_testset.csv', names=['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
test_user_id = test['user_id'].values

In [116]:
final_user_list = []
final_user_list.extend(user_list_denoise)
final_user_list.extend(test_user_id)
user_list_denoise = list(set(final_user_list))

In [117]:
df = df[df.UserId.isin(user_list_denoise)]

In [118]:
df["rating"] = df["Click"] + df["Purchase"] * 5

In [119]:
df.shape

(389923, 5)

In [56]:
df = df[df["rating"] >= 0.5]

In [57]:
df.shape

(389923, 5)

In [58]:
def sliding_window(data: list, n=1, stride=1):
    if len(data) < n:
        yield data
    else:
        for i in range(0, len(data) - n + 1, stride):
            yield data[i:i+n]

In [59]:
window_slicing = df.groupby("UserId").agg({"ItemId": list})
window_slicing["ItemId"] = window_slicing["ItemId"].apply(lambda x : [i for i in sliding_window(x, n=50, stride=10)])
window_slicing = window_slicing.explode("ItemId").reset_index().reset_index()
window_slicing = window_slicing.rename(columns={"UserId" : "TrueUser"})
window_slicing = window_slicing.rename(columns={"index": "UserId"})
window_slicing = window_slicing.explode("ItemId")
# window_slicing["rating"] = window_slicing["Click"] + window_slicing["Purchase"] * 5

In [60]:
# Merge df and window_slicing on 'UserId' and 'ItemId'
merged_df = window_slicing.merge(df[['UserId', 'ItemId', 'rating']], 
                                 left_on=['TrueUser', 'ItemId'], 
                                 right_on=['UserId', 'ItemId'], 
                                 how='left')

# Assign the 'rating' column from the merged DataFrame to window_slicing
window_slicing['rating'] = merged_df['rating']


In [61]:
window_slicing

,UserId,TrueUser,ItemId,rating
0,0,00CqJFh8ey,WCJ3v3bGfi,1
0,0,00CqJFh8ey,ERHK6OW0BA,1
0,0,00CqJFh8ey,v5DrRkI5jd,1
0,0,00CqJFh8ey,7o8G2oZDME,1
0,0,00CqJFh8ey,d6qTUg1iUa,1
...,...,...,...,...
43361,43361,zzyfHckCgU,IOqkRNQPUV,1
43361,43361,zzyfHckCgU,ZkBDQumm61,1
43361,43361,zzyfHckCgU,BRpJjYD0XK,1
43361,43361,zzyfHckCgU,K0oduAO6B2,1


In [62]:
window_slicing50 = window_slicing.copy(deep=True)

In [63]:
window_slicing = df.groupby("UserId").agg({"ItemId": list})
window_slicing["ItemId"] = window_slicing["ItemId"].apply(lambda x : [i for i in sliding_window(x, n=20, stride=10)])
window_slicing = window_slicing.explode("ItemId").reset_index().reset_index()
window_slicing = window_slicing.rename(columns={"UserId" : "TrueUser"})
window_slicing = window_slicing.rename(columns={"index": "UserId"})
window_slicing = window_slicing.explode("ItemId")
# window_slicing["rating"] = 1

In [64]:
# Merge df and window_slicing on 'UserId' and 'ItemId'
merged_df = window_slicing.merge(df[['UserId', 'ItemId', 'rating']], 
                                 left_on=['TrueUser', 'ItemId'], 
                                 right_on=['UserId', 'ItemId'], 
                                 how='left')

# Assign the 'rating' column from the merged DataFrame to window_slicing
window_slicing['rating'] = merged_df['rating']


In [65]:
window_slicing20 = window_slicing.copy(deep=True)

In [76]:
window_slicing20['rating'].unique()

array([1, 6])

In [96]:
from more_itertools import locate
sessions = []
session_id = 0
for _, i in df.groupby("UserId").agg({"ItemId": list, "Purchase": list}).reset_index().iterrows():
    userid = i["UserId"]
    for l in (list(locate(i.Purchase))):
        start = max(0, l-10)
        end = min(len(i["ItemId"]), l+10)
        sessions.append((userid, session_id, i["ItemId"][start:end]))

        # sessions.append((userid, session_id, i["ItemId"][l-10:l+10]))
        session_id+=1

sessions_df = pd.DataFrame(sessions, columns = ["TrueUser", "UserId", "ItemId"])
sessions_df = sessions_df.explode("ItemId")
# sessions_df["rating"] = 1

In [97]:
# Merge df and window_slicing on 'UserId' and 'ItemId'
merged_df = sessions_df.merge(df[['UserId', 'ItemId', 'rating']], 
                                 left_on=['TrueUser', 'ItemId'], 
                                 right_on=['UserId', 'ItemId'], 
                                 how='left')

# Assign the 'rating' column from the merged DataFrame to window_slicing
sessions_df['rating'] = merged_df['rating']


In [99]:
raw_df = df.copy(deep=True)
raw_df["TrueUser"] = raw_df["UserId"]

In [87]:
# items_less_than_3_users = df["ItemId"].value_counts()
# items_less_than_3_users = items_less_than_3_users[items_less_than_3_users >= 3]
# items_less_than_3_users = df.merge(items_less_than_3_users, on="ItemId", how="inner")
# items_less_than_3_users["TrueUser"] = items_less_than_3_users["UserId"]

In [127]:
raw_df

,UserId,ItemId,Click,Purchase,rating,TrueUser
0,tyviMi4b8Q,tyviMi4b8Q,1,0,1,tyviMi4b8Q
1,tyviMi4b8Q,t3fB9Nq1VY,1,0,1,tyviMi4b8Q
2,tyviMi4b8Q,X7ZdDwPBKM,1,0,1,tyviMi4b8Q
3,tyviMi4b8Q,r4bieaZn66,1,0,1,tyviMi4b8Q
4,tyviMi4b8Q,cRl2I3cVGB,1,0,1,tyviMi4b8Q
...,...,...,...,...,...,...
389918,16jkTS9Vj2,5wWKMYcpiw,1,0,1,16jkTS9Vj2
389919,16jkTS9Vj2,lHTwXgiMo7,1,0,1,16jkTS9Vj2
389920,16jkTS9Vj2,ZBmAO23TI8,1,0,1,16jkTS9Vj2
389921,16jkTS9Vj2,h9tLDUYcw6,1,0,1,16jkTS9Vj2


In [110]:
def get_var_name(var):
    for name, value in globals().items():
        if value is var:
            return name

def data_condat(*data_list):
    full = []
    for data in data_list:
        f = data.groupby("UserId").agg({"ItemId": list, "TrueUser": "max"})
        f["origin"] = get_var_name(data)
        full.append(f)


    full_df = pd.concat(full, ignore_index=True)
    full_df = full_df.drop_duplicates(subset="ItemId")
    full_df = full_df.reset_index().rename(columns={"index": "UserId"})
    full_df = full_df.explode("ItemId")

    return full_df

In [128]:
df_all = data_condat(window_slicing20, window_slicing50, sessions_df)

In [129]:
df_all

,UserId,ItemId,TrueUser,origin
0,0,WCJ3v3bGfi,00CqJFh8ey,window_slicing20
0,0,ERHK6OW0BA,00CqJFh8ey,window_slicing20
0,0,v5DrRkI5jd,00CqJFh8ey,window_slicing20
0,0,7o8G2oZDME,00CqJFh8ey,window_slicing20
0,0,d6qTUg1iUa,00CqJFh8ey,window_slicing20
...,...,...,...,...
66378,107208,7NNbWqcna5,zzReTyUNBb,sessions_df
66378,107208,m8B1qRL7Mn,zzReTyUNBb,sessions_df
66378,107208,sZETWbLhFP,zzReTyUNBb,sessions_df
66378,107208,76s0FMHIw0,zzReTyUNBb,sessions_df


In [130]:
# Merge df and window_slicing on 'UserId' and 'ItemId'
merged_df = df_all.merge(df[['UserId', 'ItemId', 'rating']], 
                                 left_on=['TrueUser', 'ItemId'], 
                                 right_on=['UserId', 'ItemId'], 
                                 how='left')

# Assign the 'rating' column from the merged DataFrame to window_slicing
df_all['rating'] = merged_df['rating']


In [133]:
df = df[['UserId', 'ItemId', 'rating']]

In [134]:
df_all = df_all[['UserId', 'ItemId', 'rating']]

In [135]:
merged_df = pd.concat([df, df_all], ignore_index=True)

In [136]:
merged_df.to_csv('multiverse_data.csv', index=None)

In [126]:
df_all.shape

(1171379, 5)